# [[데이콘] 영화 리뷰 감성 분석 PBL2](https://dacon.io/competitions/official/235995/codeshare/6529?page=1&dtype=recent)

In [1]:
!pip install konlpy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import re
import sys
import nltk
from collections import Counter
from wordcloud import WordCloud
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
nltk.download('all')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

In [2]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/train.csv')[['document', 'label']]
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/test.csv')[['document']]
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/sample_submission.csv')

In [3]:
train

,document,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...
149995,인간이 문제지.. 소는 뭔죄인가..,0
149996,평점이 너무 낮아서...,1
149997,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [4]:
test

,document
0,굳 ㅋ
1,GDNTOPCLASSINTHECLUB
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
...,...
49995,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함
49996,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO
49997,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다
49998,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네


In [5]:
# train/test 결측치 확인
print('train 결측치 개수\n',train.isnull().sum())
print(train[train['document'].isnull()])
print('\n')
print('test 결측치 개수\n',test.isnull().sum())
print(test[test['document'].isnull()])

train 결측치 개수
 document    5
label       0
dtype: int64
       document  label
25857       NaN      1
55737       NaN      1
110014      NaN      0
126782      NaN      0
140721      NaN      0


test 결측치 개수
 document    3
dtype: int64
      document
5746       NaN
7899       NaN
27097      NaN


In [6]:
# train/test 결측치 대체
train = train.fillna(method='ffill')
test = test.fillna(method='ffill')

In [7]:
# 결측치 개수 확인
print(train.isnull().sum())
print(test.isnull().sum())

document    0
label       0
dtype: int64
document    0
dtype: int64


### 1-3. Feature & Target 설정

In [8]:
train_feature = train.document
train_label = train.label

In [9]:
train_feature.head()

0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
Name: document, dtype: object

In [10]:
train_feature.tail()

149995                인간이 문제지.. 소는 뭔죄인가..
149996                      평점이 너무 낮아서...
149997    이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998        청춘 영화의 최고봉.방황과 우울했던 날들의 자화상
149999           한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, dtype: object

## 2. 텍스트 전처리
### 2-1. CountVectorizer의 이해
- 워드 임베딩 진행
  - 워드 임베딩: 컴퓨터가 계산하기 위해 문장을 숫자로 바꾸는 것(단어를 벡터로 표현)
- 본 분석에서는 CountVectorizer를 이용해 문장을 숫자로 표현
  - CountVectorizer: 입력된 문장을 **토큰화**하여 토큰의 **등장 빈도 벡터**로 바꾸는 기법


In [11]:
# 예시
from sklearn.feature_extraction.text import CountVectorizer

sample_vectorizer = CountVectorizer()
sample_text1 = ["hello, my name is dacon and I am a data scientist!"]
sample_vectorizer.fit(sample_text1) # CountVectorizer 학습

print(sample_vectorizer.vocabulary_) # sample_vectorizer가 학습한 단어 목록

{'hello': 4, 'my': 6, 'name': 7, 'is': 5, 'dacon': 2, 'and': 1, 'am': 0, 'data': 3, 'scientist': 8}


길이가 1개인 단어들과 특수문자는 의미가 적다고 판단하여 제외되었음

In [12]:
# 다른 문장을 활용해 벡터로 transform 진행
sample_text2 = ["you are learning dacon data science"]

sample_vector = sample_vectorizer.transform(sample_text2)
print(sample_vector.toarray())

[[0 0 1 1 0 0 0 0 0]]


↑ transform의 결과, 단어들의 출현 빈도로 이루어진 크기 9의 벡터 출력됨

이 벡터를 BoW라고 하는데, BoW란 CountVectorizer로 변환된 단어의 집합을 의미함

sample_text2에는 dacon과 data가 1번씩 출현했기에 해당 인덱스 값이 1로 표현되었고, 등장하지 않은 단어들은 0으로 표시됨

In [13]:
sample_text3 = ["you are learning dacon data science with movie data"]

sample_vector = sample_vectorizer.transform(sample_text3)
print(sample_vector.toarray())

[[0 0 1 2 0 0 0 0 0]]


"dacon" 단어가 1번, "data" 단어가 2번 출현했기에 "dacon" 단어에 해당하는 인덱스에는 1의 값이, "data" 에 해당하는 인덱스에는 2의 값이 할당됨

CountVectorizer 를 하나의 문장을 사용해 학습 시켰기에 생성된 Vocab과 BoW 의 크기는 모두 9였음

큰 데이터를 사용해 CountVectorizer를 학습시키면 Vocab과 BoW의 크기는 증가할 것

In [14]:
vectorizer = CountVectorizer() # countvectorizer 생성
vectorizer.fit(train_feature) # countvectorizer 학습(fit))
train_X = vectorizer.transform(train_feature) # transform

In [15]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(train_X, train_label)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

위에서 train 데이터로 학습한 모델로 test data를 넣어 예측값을 도출함

fit_transform() 매서드는 학습을 위한 train data에 사용되고 test data에는 transform() 매서드만 사용해야 함

test data에도 fit_transform() 매서드를 사용하는 것은 모델이 test data에 대해서도 학습을 한다는 것을 의미함 → No!!

In [16]:
test_X_vect = vectorizer.transform(test['document']) # test data는 transform() 매서드만 적용

pred = model.predict(test_X_vect)
print(pred)

[0 0 0 ... 1 0 0]


### 2-2. 데이터 정제
